In [237]:
class OZON_Meta:
    
    import requests
    import json
    import pandas as pd

    def __init__(self,client_id,api_key):
        self.client_id = client_id
        self.api_key = api_key
        self.headers = {
            'Host': 'api-seller.ozon.ru',
            'Client-Id': self.client_id,
            'Api-Key': self.api_key,
            'Content-Type': 'application/json',
        }
        
        
    def create_report_posting(self,date_start:str,date_finish:str)->str:
        """
        date_start = "2022-06-01"
        date_finish = "2022-06-02"
        """
        url_postings = 'https://api-seller.ozon.ru/v1/report/postings/create'
        
        data = {
            "filter": {
            "processed_at_from": date_start+"T00:00:00Z",
            "processed_at_to": date_finish+"T23:59:59Z",
            "delivery_schema": [
                "fbs",
                "fbo",
                "crossborder"],
                }
            }
        
        response = self.requests.post(url_postings,data=self.json.dumps(data), headers=self.headers)
        print(response.text)
        
        return response.json()['result']['code']
        
    def get_json_report(self,UUID:str):
        import time
        
        data = {"code": UUID}
        url_report = 'https://api-seller.ozon.ru/v1/report/info'

        
        for i in range(0,30):
            time.sleep(10)
            print(i, end=', ')
            response = self.requests.post(url_report,data=self.json.dumps(data), headers=self.headers)
            response_data = response.json()
            if response_data['result']['status'] == 'success':
                break
                
        return response_data['result']['file']
    
    def parse_csv(self, csv_file:str):
        
        data_text = self.requests.get(csv_file, headers = self.headers)
        text_csv = data_text.text
        data_parse = [ elem.split(';') for elem in text_csv.split('\n')]
        
        df = self.pd.DataFrame(data_parse)
        df =df.rename(columns=df.iloc[0])
        df = df.reindex(df.index.drop(0))
        df.columns = df.columns.str.replace('"','')
        df = df.dropna(subset = ['Номер отправления'])
        
        return df
    
    

# Данные

In [238]:
client_id_bot = ''
api_key_bot = ''

client_id_abumba = ''
api_key_abumba = ''

# Отчет об отправлениях

In [336]:
start_date = "2022-06-01"

In [337]:
def finish_month(date:str):
    """формат 2021-01-01"""
    from datetime import timedelta
    import datetime
    
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    finish_date = datetime.date(dt.year,dt.month+1,1)-timedelta(days=1)
    return finish_date.strftime("%Y-%m-%d")

In [338]:
finish_date=finish_month(start_date)

In [339]:
finish_date

'2022-06-30'

In [340]:
def program(client_id,api_key,start_date,finish_date):
    OZ_Meta_compamy = OZON_Meta(
            client_id = client_id,
            api_key = api_key)
    
    
    from datetime import datetime,timedelta
    import time
    import pandas as pd

    
    code_report = OZ_Meta_compamy.create_report_posting(start_date,finish_date)
    csv_file_path = OZ_Meta_compamy.get_json_report(code_report)
    
    df = OZ_Meta_compamy.parse_csv(csv_file_path)
    for col in df.columns:
        df[col] = df[col].str.replace('"','')
        
    df['Дата и время заказа'] = pd.to_datetime(df['Дата и время заказа'])
    df['Дата отгрузки'] = pd.to_datetime(df['Дата и время заказа'])
    df['Стоимость'] = df['Стоимость'].astype(float)
    df['Цена'] = df['Цена'].astype(float)
    df['Количество'] = df['Количество'].astype(int)
    
    df_order_columns = ['﻿Номер заказа', 'Номер отправления', 'Дата и время заказа',
       'Дата отгрузки', 'Статус', 'Стоимость', 'Наименование товара',
       'Озон ID', 'Артикул', 'Цена', 'Количество']
    df = df[df_order_columns]
    
    df = df.rename(columns = {
        '﻿Номер заказа':'num_order',
        'Номер отправления':'num_shipment',
        'Дата и время заказа':'date_order',
        'Дата отгрузки':'date_shipment',
        'Статус':'status_order',
        'Стоимость':'cost',
        'Наименование товара':'name',
        'Озон ID':'ozon_id',
        'Артикул':'article',
        'Цена':'price',
        'Количество':'order_count',

    })
    
    df['indexed_row'] = df['num_shipment']+"_"+df['ozon_id']
    print('success')
    return df

In [341]:
df_BOT = program(
    client_id = client_id_bot,
    api_key = api_key_bot,
    start_date=start_date,
    finish_date=finish_date
)

{"result":{"code":"0a41d623-e98a-4502-89fa-472b7e8e4a51"}}
0, 1, success


In [342]:
df_ABUMBA = program(
    client_id = client_id_abumba,
    api_key = api_key_abumba,
    start_date=start_date,
    finish_date=finish_date
)

{"result":{"code":"ef4e7f8d-903f-4f76-998b-82fe0d957570"}}
0, success


In [343]:
import pandas as pd
df_all = pd.concat([df_BOT,df_ABUMBA])

In [344]:
df_all.head(2)

,num_order,num_shipment,date_order,date_shipment,status_order,cost,name,ozon_id,article,price,order_count,indexed_row
1,05817831-0242,05817831-0242-8,2022-06-20 09:28:51,2022-06-20 09:28:51,cancelled,1745.0,Детские солнцезащитные очки Babiators Aviator ...,150264966,BAB-015,1745.0,1,05817831-0242-8_150264966
2,58221142-0039,58221142-0039-1,2022-06-01 02:03:33,2022-06-01 02:03:33,delivered,455.0,Соска антиколиковая Twistshake в наборе из 2 ш...,171609390,78021,455.0,1,58221142-0039-1_171609390


# Обработка данных

In [345]:
class Product(OZON_Meta):
    
    def get_list_sku(self,last_id,status_archive):
        url_get_sku = 'https://api-seller.ozon.ru/v2/product/list'
        visibility = "ALL"
        if status_archive == True:
            visibility = "ARCHIVED"
        data = {
            "filter":{
                "visibility": visibility,
            },
            "limit": "1000",
            "last_id":last_id
        }
        
        response = self.requests.post(url_get_sku,data=self.json.dumps(data), headers=self.headers)
        return response.json()
    
    def get_list_circle(self,status_archive):
        last_id = ""
        data_json_product = []
        for i in range(20):
            data_ = self.get_list_sku(last_id,status_archive=False)
            if data_['result']['last_id'] == '':
                break

            data_json_product.extend(data_['result']['items'])
            last_id = data_['result']['last_id']
            
        return data_json_product
    
    def get_all_product(self):
        data_ = []
        
        #data_.extend(self.get_list_circle(status_archive=True))
        #архивные товара в API OZON не работают
        data_.extend(self.get_list_circle(status_archive=False))
        
        return data_
    
    
    def get_product_info(self,product_id):
        data = {
                "product_id": product_id,
            }
        
        url_get_sku_info = 'https://api-seller.ozon.ru/v2/product/info'
        
        response = self.requests.post(url_get_sku_info,data=self.json.dumps(data), headers=self.headers)
        
        return response.json()
    
    def get_barcode(self,product_id):

        try:
            barcode = self.get_product_info(product_id)['result']['barcode']
            
        except:
            barcode = "---"
        
        return barcode

In [346]:
def get_all_product(client_id,api_key):
    prod_obj = Product(client_id = client_id,
            api_key = api_key)
    
    data_ = prod_obj.get_all_product()
    df = pd.DataFrame(data_)
    df['product_id'] = df['product_id'].astype(str)
    df['barcode'] = df['product_id'].apply(prod_obj.get_barcode)
    
    return df

In [347]:
df_product_BOT = get_all_product(client_id_bot,api_key_bot)
df_product_abumba = get_all_product(client_id_abumba,api_key_abumba)

In [348]:
df_all_product = pd.concat([df_product_BOT,df_product_abumba])

In [349]:
df_all_product = df_all_product[df_all_product['barcode']!='']

In [350]:
del df_all_product['product_id']

In [351]:
df_all_product.drop_duplicates(ignore_index=True,inplace=True)

**Работа с пропусками**

In [321]:
df_total = df_all.merge(right = df_all_product,how = 'left',left_on='article',right_on='offer_id')

In [322]:
path_oz_ms = '/Users/Aleksey/Documents/Файлы для БД/OZ_MS.xlsx'

In [323]:
df_ms_oz_code = pd.read_excel(path_oz_ms,dtype = {'ozon_id': 'str', 
     'offer_id': 'str',
      'barcode_ms': 'str',                                           })

In [324]:
df_ms_oz_code = df_ms_oz_code[['ozon_id','offer_id','barcode_ms']]

In [325]:
df_total = df_total.merge(right = df_ms_oz_code,on = 'ozon_id',how = 'left',suffixes = ('_oz','_oz_2'))
df_total.loc[df_total['barcode_ms'].isnull(),'barcode_ms'] = df_total.loc[df_total['barcode_ms'].isnull(),'barcode']
df_total.loc[df_total['offer_id_oz_2'].isnull(),'offer_id_oz_2'] = df_total.loc[df_total['offer_id_oz_2'].isnull(),'offer_id_oz']

In [326]:
df_total[df_total['barcode_ms']==""]

,num_order,num_shipment,date_order,date_shipment,status_order,cost,name,ozon_id,article,price,order_count,indexed_row,offer_id_oz,barcode,offer_id_oz_2,barcode_ms


In [327]:
df_total[df_total['barcode_ms']=="---"]

,num_order,num_shipment,date_order,date_shipment,status_order,cost,name,ozon_id,article,price,order_count,indexed_row,offer_id_oz,barcode,offer_id_oz_2,barcode_ms


In [328]:
df_total[df_total['barcode_ms'].isna()][['ozon_id','article','num_shipment']].drop_duplicates()

,ozon_id,article,num_shipment


In [329]:
del df_total['barcode']
del df_total['offer_id_oz']

In [330]:
df_total.head(2)

,num_order,num_shipment,date_order,date_shipment,status_order,cost,name,ozon_id,article,price,order_count,indexed_row,offer_id_oz_2,barcode_ms
0,23548512-0089,23548512-0089-3,2022-07-01 00:06:39,2022-07-01 00:06:39,delivered,685.0,Антиколиковая бутылочка Twistshake для кормлен...,149359464,78255,685.0,1,23548512-0089-3_149359464,78255,7350083122551
1,44813635-0062,44813635-0062-15,2022-07-01 01:35:02,2022-07-01 01:35:02,delivered,650.0,Антиколиковая бутылочка Twistshake для кормлен...,149359459,78250,650.0,1,44813635-0062-15_149359459,78250,7350083122506


# Выгрузка отчета в MySQL 

In [331]:
import sqlalchemy
import pandas as pd

database_username = ''
database_password = ''
database_ip       = ''
database_name     = ''
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [332]:
name_main_table_sale = 'ozon_order_1'

In [333]:
def join_table(name_table,data):
    name_temp = name_table+'_temp'
    for i in range(0,len(data),50):
        with database_connection.begin() as connection:
            data.iloc[i:i+50].to_sql(con=connection, name=name_temp, if_exists='append')
            if i%1000==0:
                print(i, end=', ')
                
    SQL_DROP = f"""DELETE FROM {name_table} 
    where {name_table}.indexed_row in (select indexed_row from {name_temp})"""
    
    result = database_connection.execute(SQL_DROP)
    result = database_connection.execute(f"INSERT INTO {name_table} SELECT * FROM {name_temp}")
    result = database_connection.execute(f"DROP TABLE {name_temp}")

In [334]:
join_table(name_main_table_sale,df_total)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 

In [335]:
database_connection.dispose()

Вариант когда нету таблицы (для тестов)

In [183]:
# def insert_table(name_table,data):
#     for i in range(0,len(data),50):
#         with database_connection.begin() as connection:
#             data.iloc[i:i+50].to_sql(con=connection, name=name_table, if_exists='append')
#             if i%1000==0:
#                 print(i, end=', ')

In [184]:
# insert_table(name_main_table_sale,df_total)

0, 1000, 2000, 

In [185]:
# database_connection.dispose()